In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Veri setini yükle
train_data = pd.read_csv('/home/kpo/Downloads/mnist-deep-learning/data/mnist_train.csv')
test_data = pd.read_csv('/home/kpo/Downloads/mnist-deep-learning/data/mnist_test.csv')

# Veriyi küçült
train_data = train_data.sample(frac=0.1, random_state=1)
test_data = test_data.sample(frac=0.1, random_state=1)

# Bağımsız değişkenleri ve hedef değişkeni ayır
X_train = train_data.drop('label', axis=1).values
y_train = to_categorical(train_data['label'].values)

X_test = test_data.drop('label', axis=1).values
y_test = to_categorical(test_data['label'].values)

# Veriyi normalize et
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Giriş boyutlarını yeniden boyutlandır ve genişlet
X_train_reshaped = X_train.reshape(-1, 28, 28, 1)
X_test_reshaped = X_test.reshape(-1, 28, 28, 1)
X_train_resized = tf.image.resize(X_train_reshaped, (224, 224))
X_test_resized = tf.image.resize(X_test_reshaped, (224, 224))
X_train_resized_rgb = np.stack((X_train_resized[:, :, :, 0],) * 3, axis=-1)
X_test_resized_rgb = np.stack((X_test_resized[:, :, :, 0],) * 3, axis=-1)

# MobileNet modelini oluştur
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_mobilenet = models.Sequential()
model_mobilenet.add(base_model)
model_mobilenet.add(layers.Flatten())
model_mobilenet.add(layers.Dense(256, activation='relu'))
model_mobilenet.add(layers.Dropout(0.5))
model_mobilenet.add(layers.Dense(10, activation='softmax'))
model_mobilenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğit
model_mobilenet.fit(X_train_resized_rgb, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Modeli değerlendir
y_pred_mobilenet = model_mobilenet.predict(X_test_resized_rgb)
y_pred_mobilenet_classes = np.argmax(y_pred_mobilenet, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy_mobilenet = accuracy_score(y_test_classes, y_pred_mobilenet_classes)
precision_mobilenet = precision_score(y_test_classes, y_pred_mobilenet_classes, average='weighted')
recall_mobilenet = recall_score(y_test_classes, y_pred_mobilenet_classes, average='weighted')
f1_mobilenet = f1_score(y_test_classes, y_pred_mobilenet_classes, average='weighted')

print(f"Accuracy: {accuracy_mobilenet}")
print(f"Precision: {precision_mobilenet}")
print(f"Recall: {recall_mobilenet}")
print(f"F1 Score: {f1_mobilenet}")


2023-12-23 11:58:10.130879: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-23 11:58:10.465834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-23 11:58:10.465900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-23 11:58:10.528263: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-23 11:58:10.662368: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-23 11:58:10.663986: I tensorflow/core/platform/cpu_feature_guard.cc:1

Epoch 1/5
75/75 [==============================] - 314s 4s/step - loss: 2.0074 - accuracy: 0.8494 - val_loss: 3.0423 - val_accuracy: 0.5592
Epoch 2/5
75/75 [==============================] - 285s 4s/step - loss: 0.1923 - accuracy: 0.9504 - val_loss: 0.2003 - val_accuracy: 0.9650
Epoch 3/5
75/75 [==============================] - 306s 4s/step - loss: 0.1555 - accuracy: 0.9631 - val_loss: 0.1085 - val_accuracy: 0.9725
Epoch 4/5
75/75 [==============================] - 323s 4s/step - loss: 0.1281 - accuracy: 0.9700 - val_loss: 0.1224 - val_accuracy: 0.9750
Epoch 5/5
32/32 [==============================] - 12s 348ms/step
MobileNet Model:
Accuracy: 0.956
Precision: 0.958169132677347
Recall: 0.956
F1 Score: 0.956035355296241
